In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
import warnings
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')
submission_df = pd.read_csv('sample_submission.csv')

In [26]:
train_df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.00,3,0.00,2,1.00,0.00,"181,449.97",0
1,1,15749177,Okwudiliolisa,627,France,Male,33.00,1,0.00,2,1.00,1.00,"49,503.50",0
2,2,15694510,Hsueh,678,France,Male,40.00,10,0.00,2,1.00,0.00,"184,866.69",0
3,3,15741417,Kao,581,France,Male,34.00,2,"148,882.54",1,1.00,1.00,"84,560.88",0
4,4,15766172,Chiemenam,716,Spain,Male,33.00,5,0.00,2,1.00,1.00,"15,068.83",0


In [7]:
submission_df.head()

,id,Exited
0,165034,0.5
1,165035,0.5
2,165036,0.5
3,165037,0.5
4,165038,0.5


In [27]:
test_df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.00,2,0.00,2,0.00,1.00,"160,976.75"
1,165035,15782418,Nott,683,France,Female,46.00,2,0.00,1,1.00,0.00,"72,549.27"
2,165036,15807120,K?,656,France,Female,34.00,7,0.00,2,1.00,0.00,"138,882.09"
3,165037,15808905,O'Donnell,681,France,Male,36.00,8,0.00,1,1.00,0.00,"113,931.57"
4,165038,15607314,Higgins,752,Germany,Male,38.00,10,"121,263.62",1,1.00,0.00,"139,431.00"


In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               165034 non-null  int64  
 1   CustomerId       165034 non-null  int64  
 2   Surname          165034 non-null  object 
 3   CreditScore      165034 non-null  int64  
 4   Geography        165034 non-null  object 
 5   Gender           165034 non-null  object 
 6   Age              165034 non-null  float64
 7   Tenure           165034 non-null  int64  
 8   Balance          165034 non-null  float64
 9   NumOfProducts    165034 non-null  int64  
 10  HasCrCard        165034 non-null  float64
 11  IsActiveMember   165034 non-null  float64
 12  EstimatedSalary  165034 non-null  float64
 13  Exited           165034 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 17.6+ MB


In [29]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,"165,034.00","82,516.50","47,641.36",0.00,"41,258.25","82,516.50","123,774.75","165,033.00"
CustomerId,"165,034.00","15,692,005.02","71,397.82","15,565,701.00","15,633,141.00","15,690,169.00","15,756,824.00","15,815,690.00"
CreditScore,"165,034.00",656.45,80.10,350.00,597.00,659.00,710.00,850.00
Age,"165,034.00",38.13,8.87,18.00,32.00,37.00,42.00,92.00
Tenure,"165,034.00",5.02,2.81,0.00,3.00,5.00,7.00,10.00
Balance,"165,034.00","55,478.09","62,817.66",0.00,0.00,0.00,"119,939.52","250,898.09"
NumOfProducts,"165,034.00",1.55,0.55,1.00,1.00,2.00,2.00,4.00
HasCrCard,"165,034.00",0.75,0.43,0.00,1.00,1.00,1.00,1.00
IsActiveMember,"165,034.00",0.50,0.50,0.00,0.00,0.00,1.00,1.00
EstimatedSalary,"165,034.00","112,574.82","50,292.87",11.58,"74,637.57","117,948.00","155,152.47","199,992.48"
